In [ ]:
import sys
import os

# Determine the absolute path to the src directory
src_path = os.path.abspath(os.path.join(os.getcwd(), 'src'))

# Add src_path to sys.path if it's not already present
if src_path not in sys.path:
    sys.path.insert(0, src_path)

In [ ]:
import typing
import torch
from typing import Iterator
import re
import numpy as np
import pandas as pd

import os
import random

from tabpfn_extensions import TabPFNRegressor, TabPFNClassifier
from tabpfn_extensions.embedding import TabPFNEmbedding
from tabpfn.constants import (
    NA_PLACEHOLDER,
)

# import to figure out what is going on with the data
from tabpfn.utils import validate_X_predict, _fix_dtypes, _process_text_na_dataframe

from tabpfn.config import ModelInterfaceConfig
from tabpfn.preprocessing import EnsembleConfig

from src.data_constants import *
from src.data_utils import load_data_for_tabPFN


In [ ]:
# Set the random seed for reproducibility
s = 42
np.random.seed(s)
random.seed(s)

# load the dataset
f = 'data/processed/vitals_train_data.csv'
if f.endswith(".csv"):
    data, ids = load_data_for_tabPFN(f)
    print(f"Loaded data from {f}")

In [ ]:
config = ModelInterfaceConfig(
    FEATURE_SHIFT_METHOD = None,
    CLASS_SHIFT_METHOD = None,
    FINGERPRINT_FEATURE = False,
)

In [ ]:
categorical_features_ = [data.columns.get_loc(col) for col in categorical_features if col in data.columns]

In [ ]:
categorical_features_

In [ ]:
reg = TabPFNRegressor(random_state=s,  
                      categorical_features_indices=categorical_features_,
                      inference_config=config,
                      n_estimators = 1
                     )

In [ ]:
reg.fit(data, np.arange(data.shape[0]))

In [ ]:
x = data

In [ ]:
x.dtypes

In [ ]:
X = validate_X_predict(x, reg)

In [ ]:
X = _fix_dtypes(X, cat_indices=reg.categorical_features_indices)

In [ ]:
X = _process_text_na_dataframe(X, ord_encoder=reg.preprocessor_)

In [ ]:
X.shape

In [ ]:
print(x.columns[x.isna().all(axis=0)].values[0])

In [ ]:
reg.executor_.ensemble_configs

In [ ]:
pipeline = reg.executor_.preprocessors[0]

In [ ]:
pipeline

In [ ]:
pipeline[1].get_column_types(X)

In [ ]:
pipeline[1].get_adaptive_preprocessors()["adaptive"]

In [ ]:
pipeline[1].get_all_preprocessors(100)["safepower"]

In [ ]:
pipeline[1].subsample_features

In [ ]:
pipeline[1].apply_to_categorical

In [ ]:
pipeline[2].categorical_transformer_

In [ ]:
pd.DataFrame(X)

In [ ]:
inspecting = X.copy()

In [ ]:
pd.DataFrame(pipeline[0].transform(inspecting).X)

In [ ]:
pd.DataFrame(pipeline[1].transform(inspecting).X)

In [ ]:
pd.DataFrame(pipeline[2].transform(inspecting).X)

In [ ]:
pd.DataFrame(reg.executor_.preprocessors[0].transform(X).X)

In [ ]:
print(dir(pipeline[3]))

In [ ]:
print(pipeline[3].shuffle_method)

In [ ]:
shuffle_idx = [idx for i in [col for _, _, col in reg.preprocessor_.transformers_] for idx in i]

In [ ]:
sort_idx = np.argsort(shuffle_idx)

In [ ]:
x_in = tran[:, sort_idx]

In [ ]:
pd.DataFrame(x_in)

In [ ]:
data

In [ ]:
x

In [ ]:
pd.DataFrame(tran)